# Diffusion of a Gaussian function, Failure due to poor mesh



Solving the diffusion of a Gaussian hill. However the mesh is poor nx, ny = 2
\begin{align}
    u_0(x,y)&= e^{-ax^2-ay^2}
\end{align}
for $a=5$ on the domain $[-2,2]\times[-2,2]$. For this problem we will use homogeneous Dirichlet boundary conditions ($u_D=0$).

The first difference from the previous problem is that we are not using a unit square. We create the rectangular domain with `dolfinx.mesh.create_rectangle`.

In [1]:
import matplotlib as mpl  # Imports Matplotlib for plotting capabilities.
import pyvista  # Imports PyVista for 3D visualization.
import ufl  # Imports UFL for defining finite element forms.
import numpy as np  # Imports NumPy for numerical computations.

from petsc4py import PETSc  # Imports PETSc for linear algebra solvers.
from mpi4py import MPI  # Imports MPI for parallel computing.

from dolfinx import fem, mesh, io, plot  # Imports FEniCSx modules for finite element computations.
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc  # Imports PETSc utilities for finite element assembly.

# Define temporal parameters
t = 0  # Sets the initial time to 0.
T = 1.0  # Sets the final simulation time to 1.0.
num_steps = 50  # Defines 50 time steps for the simulation.
dt = T / num_steps  # Computes the time step size.

# Define mesh
nx, ny = 2, 2  # Specifies mesh resolution as 50x50.
domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([-2, -2]), np.array([2, 2])], [nx, ny], mesh.CellType.triangle)  # Generates a triangular mesh for the domain.
V = fem.functionspace(domain, ("Lagrange", 1))  # Creates a P1 function space for the solution.

Note that we have used a much higher resolution than before to better resolve features of the solution.
We also easily update the intial and boundary conditions. Instead of using a class to define the initial condition, we simply use a function

In [2]:
# Create initial condition
def initial_condition(x, a=5):  # Defines the Gaussian initial condition
    return np.exp(-a * (x[0]**2 + x[1]**2))  # Computes the Gaussian value at point x

u_n = fem.Function(V)  # Creates a function for the previous time step solution
u_n.name = "u_n"  # Names the function for easy identification
u_n.interpolate(initial_condition)  # Sets the initial condition by interpolation

# Create boundary condition
fdim = domain.topology.dim - 1  # Determines the boundary facet dimension
boundary_facets = mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))  # Identifies all boundary facets
bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)  # Defines homogeneous Dirichlet boundary conditions

## Time-dependent output
To visualize the solution in an external program such as Paraview, we create a an `XDMFFile` which we can store multiple solutions in. The main advantage with an XDMFFile is that we only need to store the mesh once and that we can append multiple solutions to the same grid, reducing the storage space.
The first argument to the XDMFFile is which communicator should be used to store the data. As we would like one output, independent of the number of processors, we use the `COMM_WORLD`. The second argument is the file name of the output file, while the third argument is the state of the file,
this could be read (`"r"`), write (`"w"`) or append (`"a"`).

In [3]:
xdmf = io.XDMFFile(domain.comm, "diffusion.xdmf", "w")  # Creates an XDMF file for output using the domain’s communicator.
xdmf.write_mesh(domain)  # Writes the mesh to the XDMF file for visualization.

# Define solution variable, and interpolate initial solution for visualization in Paraview
uh = fem.Function(V)  # Creates a function to hold the solution in space V.
uh.name = "uh"  # Names the function for easy identification.
uh.interpolate(initial_condition)  # Sets the initial condition on the solution function.
xdmf.write_function(uh, t)  # Writes the initial solution to the XDMF file at time t.

## Variational problem and solver
As in the previous example, we prepare objects for time dependent problems, such that we do not have to recreate data-structures.

In [4]:
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)  # Trial function (solution) and test function from space V.
f = fem.Constant(domain, PETSc.ScalarType(0))      # Source term, set to 0 (no external sources).
a = u * v * ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx  # Bilinear form: mass + diffusion terms.
L = (u_n + dt * f) * v * ufl.dx                    # Linear form: previous solution + source contribution.

## Preparing linear algebra structures for time dependent problems
We note that even if `u_n` is time dependent, we will reuse the same function for `f` and `u_n` at every time step. We therefore call `dolfinx.fem.form` to generate assembly kernels for the matrix and vector.

In [5]:
bilinear_form = fem.form(a)  # Assembles the system matrix
linear_form = fem.form(L)    # Assembles the right-hand side vector

We observe that the left hand side of the system, the matrix $A$ does not change from one time step to another, thus we only need to assemble it once. However, the right hand side, which is dependent on the previous time step `u_n`, we have to assemble it every time step. Therefore, we only create a vector `b` based on `L`, which we will reuse at every time step.

In [6]:
A = assemble_matrix(bilinear_form, bcs=[bc])    #Assembles the system matrix from the bilinear form, applying Dirichlet boundary conditions.
A.assemble()    #Completes the matrix assembly process.
b = create_vector(linear_form)      #Initializes the RHS vector for the linear form.

## Using petsc4py to create a linear solver
As we have already assembled `a` into the matrix `A`, we can no longer use the `dolfinx.fem.petsc.LinearProblem` class to solve the problem. Therefore, we create a linear algebra solver using PETSc, assign the matrix `A` to the solver, and choose the solution strategy.

In [7]:
solver = PETSc.KSP().create(domain.comm)  # Creates KSP solver with domain communicator
solver.setOperators(A)                    # Sets matrix A for solver
solver.setType(PETSc.KSP.Type.PREONLY)    # Uses preconditioner-only solver
solver.getPC().setType(PETSc.PC.Type.LU)  # Sets LU factorization preconditioner

## Visualization of time dependent problem using pyvista
We use the DOLFINx plotting functionality, which is based on pyvista to plot the solution at every $15$th time step. We would also like to visualize a colorbar reflecting the minimal and maximum value of $u$ at each time step. We use the following convenience function `plot_function` for this:

In [8]:
pyvista.start_xvfb()                      # Starts virtual display for rendering on headless systems

grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V))  # Creates unstructured grid from VTK mesh

plotter = pyvista.Plotter()               # Initializes PyVista plotter for visualization
plotter.open_gif("u_time.gif", fps=10)    # Opens GIF file for animation at 10 frames per second

grid.point_data["uh"] = uh.x.array        # Assigns solution array to grid's point data
warped = grid.warp_by_scalar("uh", factor=1)  # Warps grid based on solution values

viridis = mpl.colormaps.get_cmap("viridis").resampled(25)  # Gets resampled viridis colormap
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2e", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)  # Defines scalar bar style
             
renderer = plotter.add_mesh(warped, show_edges=True, lighting=False,
                            cmap=viridis, scalar_bar_args=sargs,
                            clim=[0, max(uh.x.array)])  # Adds warped mesh to plot with colormap and scalar bar

## Updating the solution and right hand side per time step
To be able to solve the variation problem at each time step, we have to assemble the right hand side and apply the boundary condition before calling
`solver.solve(b, uh.x.petsc_vec)`. We start by resetting the values in `b` as we are reusing the vector at every time step.
The next step is to assemble the vector calling `dolfinx.fem.petsc.assemble_vector(b, L)`, which means that we are assembling the linear form `L(v)` into the vector `b`. Note that we do not supply the boundary conditions for assembly, as opposed to the left hand side.
This is because we want to use lifting to apply the boundary condition, which preserves symmetry of the matrix $A$ in the bilinear form $a(u,v)=a(v,u)$ without Dirichlet boundary conditions.
When we have applied the boundary condition, we can solve the linear system and update values that are potentially shared between processors.
Finally, before moving to the next time step, we update the solution at the previous time step to the solution at this time step.

In [9]:
for i in range(num_steps):
    t += dt                                             # Increments time by time step

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:                        # Accesses local form of vector b
        loc_b.set(0)                                    # Resets vector b to zero
    assemble_vector(b, linear_form)                     # Assembles right-hand side vector

    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [bilinear_form], [[bc]])           # Applies Dirichlet BC lifting
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)  # Updates ghost values
    set_bc(b, [bc])                                     # Sets Dirichlet BC on vector

    # Solve linear problem
    solver.solve(b, uh.x.petsc_vec)                     # Solves linear system
    uh.x.scatter_forward()                              # Updates solution array

    # Update solution at previous time step (u_n)
    u_n.x.array[:] = uh.x.array                         # Stores current solution for next step

    # Write solution to file
    xdmf.write_function(uh, t)                         # Writes solution to file
    # Update plot
    new_warped = grid.warp_by_scalar("uh", factor=1)    # Warps grid for visualization
    warped.points[:, :] = new_warped.points             # Updates warped grid points
    warped.point_data["uh"][:] = uh.x.array             # Updates solution data for plot
    plotter.write_frame()                              # Writes current frame to GIF
plotter.close()                                         # Closes plotter and finalizes GIF
xdmf.close()                                            # Closes output file

[gif](poor_mesh.gif)

## Animation with Paraview
We can also use Paraview to create an animation. We open the file in paraview with `File->Open`, and then press `Apply` in the properties panel.

Then, we add a time-annotation to the figure, pressing: `Sources->Alphabetical->Annotate Time` and `Apply` in the properties panel. It Is also a good idea to select an output resolution, by pressing `View->Preview->1280 x 720 (HD)`.

Then finally, click `File->Save Animation`, and save the animation to the desired format, such as `avi`, `ogv` or a sequence of `png`s. Make sure to set the frame rate to something sensible, in the range of $5-10$ frames per second.